In [1]:
%run common_imports.py

%matplotlib qt
%config InlineBackend.figure_format = 'retina'

%reload_ext autoreload
%autoreload 2

dj.config['display.limit'] = 10**3  
plt.style.use("seaborn-v0_8-talk")

os.environ["SPYGLASS_USE_TRANSACTIONS"] = "1" 
os.environ['KACHERY_API_KEY'] = "RhysjLwgmBAt2ObCyXXaDnqAv2kTdYRa"

import sys
import notebooks.sorting_multiprocessing as smp
import spyglass.spikesorting.v1 as sgs
import spikeinterface as si
import spikeinterface.widgets as sw
import spikeinterface.core as sc
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput
from spyglass.spikesorting.analysis.v1.group import SortedSpikesGroup


[2025-12-14 17:37:05,854][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-14 17:37:05,868][INFO]: Connected anirudh@172.16.102.154:3306


### Set Sort Group

In [2]:
sgc.ElectrodeGroup() & session_restrict

nwb_file_name name of the NWB file,electrode_group_name electrode group name from NWBFile,region_id,probe_id a unique ID for this probe & dynamic config,description description of electrode group,target_hemisphere
wilbur20210512_.nwb,0,7,four wire electrode,tetrode_reference,Left
wilbur20210512_.nwb,1,8,four wire electrode,tetrode,Left
wilbur20210512_.nwb,10,8,four wire electrode,tetrode,Left
wilbur20210512_.nwb,11,8,four wire electrode,tetrode,Left
wilbur20210512_.nwb,12,9,four wire electrode,tetrode,Right
wilbur20210512_.nwb,13,9,four wire electrode,tetrode,Right
wilbur20210512_.nwb,14,9,four wire electrode,tetrode,Right
wilbur20210512_.nwb,15,9,four wire electrode,tetrode,Right
wilbur20210512_.nwb,16,9,four wire electrode,tetrode,Right
wilbur20210512_.nwb,17,8,four wire electrode,tetrode,Right


In [10]:
# set sort group, using reference electrodes from the corpus callosum

references = {}
hpc_reference = "0"
for index, rows in pd.DataFrame((sgc.ElectrodeGroup() & session_restrict).fetch()).iterrows():
    e_group = rows["electrode_group_name"]
    region = rows["region_id"]
    if (region != 11):
        references.update({e_group:hpc_reference})
    else:
        references.update({e_group:-1}) # no reference for mpfc, ofc
        

sgs.SortGroup.set_group_by_shank(nwb_file_name=nwb_copy_file_name, references=references)

In [11]:
sgs.SortGroup() & session_restrict

nwb_file_name name of the NWB file,sort_group_id,sort_reference_electrode_id the electrode to use for referencing
wilbur20210512_.nwb,0,0
wilbur20210512_.nwb,1,0
wilbur20210512_.nwb,2,0
wilbur20210512_.nwb,3,0
wilbur20210512_.nwb,4,0
wilbur20210512_.nwb,5,0
wilbur20210512_.nwb,6,0
wilbur20210512_.nwb,7,0
wilbur20210512_.nwb,8,0
wilbur20210512_.nwb,9,0


### SpikeSortingRecording

In [4]:
sgs.SpikeSortingPreprocessingParameters()

preproc_param_name,preproc_params
default,=BLOB=
Hippocampups_default,=BLOB=


In [23]:
(sgs.SpikeSortingPreprocessingParameters & {"preproc_param_name": "default"}).fetch1("preproc_params")

[2025-12-12 20:56:22,329][WARNING]: MySQL server has gone away. Reconnecting to the server.


{'frequency_min': 300,
 'frequency_max': 6000,
 'margin_ms': 5,
 'seed': 0,
 'min_segment_length': 1}

In [ ]:
(sgc.IntervalList() & session_restrict).plot_intervals()

In [19]:
from itertools import product
sort_group_ids = np.arange(24, 36) # using only PFC data
interval_lists = ["02_wtrack","04_wtrack", "06_wtrack", "08_wtrack"]

for sort_group_id, interval_list in product(sort_group_ids, interval_lists): 
    key = {
        "nwb_file_name": nwb_copy_file_name,
        "sort_group_id": sort_group_id,
        "preproc_param_name": "default",
        "interval_list_name": interval_list,
        "team_name": "NeuroACT",
    }
    sgs.SpikeSortingRecordingSelection.insert_selection(key)

In [2]:
from spyglass.common.common_interval import IntervalList

base_key = {"nwb_file_name": nwb_copy_file_name}
epoch_names = ["02_wtrack", "04_wtrack", "06_wtrack", "08_wtrack"]

# Fetch valid_times for each epoch interval list
valid_times_list = []
for name in epoch_names:
    vt = (IntervalList & {**base_key, "interval_list_name": name}).fetch1("valid_times")
    valid_times_list.append(vt)

# Stack all intervals into one array
all_valid_times = np.vstack(valid_times_list)

# Insert a new combined IntervalList
IntervalList.insert1(
    {
        **base_key,
        "interval_list_name": "wtrack_all",
        "valid_times": all_valid_times,
    },
    skip_duplicates=True,
)


In [ ]:
# default parameters: bandpass filter (600 - 6000 Hz), apply whitening
module_dir = os.getcwd()         
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)
keys = (sgs.SpikeSortingRecordingSelection() - sgs.SpikeSortingRecording()).fetch("KEY")
pfc_keys = keys.copy()

procs = smp.launch_workers((sgs.SpikeSortingRecording & session_restrict), keys, n_workers=8) # run 8 parallel processes
for p in procs: p.join()


[2025-12-13 16:50:55,122][WARNING]: MySQL server has gone away. Reconnecting to the server.


### Artifact Detection

In [34]:
recording_ids = pfc_keys.copy()
for recording_id in recording_ids:
   (sgs.ArtifactDetectionSelection).insert_selection({"recording_id": recording_id["recording_id"], # skip_duplicates = True by default
                                "artifact_param_name": "default"})


[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already inserted.
[21:04:43][WARNING] Spyglass: Similar row(s) already in

In [35]:
keys = (sgs.ArtifactDetectionSelection() & pfc_keys).fetch("KEY") 
procs = smp.launch_workers(sgs.ArtifactDetection, keys, n_workers=12)
for p in procs: p.join()

[2025-12-12 21:05:01,710][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,711][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,713][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,714][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,714][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,717][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,721][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,723][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,724][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,726][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,727][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,727][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,727][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:05:01,730][INFO]: Connected anirudh@172.16.102.154:

In [37]:
sgs.ArtifactDetection() & session_restrict

artifact_id
09c4ffc8-19f3-4025-829a-660d8c4fcf2d
0fa66081-d12a-4580-ab58-a4aa9038f048
171a433f-7442-45fb-b17d-3c80ad8bc1e3
1c4f8899-3407-4647-ae97-ef9e5cb70f36
1ea1ecae-cb1e-4bf8-946f-6254ad8598f6
26d767e9-28c0-42ee-a2ae-1493cd595a74
3bd8cfa9-60b1-4436-8c0d-7b5adc765fea
3d7cbf8e-c81b-4afd-b0b0-876a377cbd01
3e98bf2a-7400-4098-ae5b-2efaa95b72fc
42eef1ad-a305-40a4-a337-2edff92550bc


### SpikeSorting

In [38]:
(sgs.SpikeSorterParameters() &  {"sorter_param_name": "pfc_default"}).fetch1()

{'sorter': 'mountainsort4',
 'sorter_param_name': 'pfc_default',
 'sorter_params': {'detect_sign': -1,
  'adjacency_radius': 115,
  'filter': False,
  'whiten': True,
  'num_workers': 4,
  'clip_size': 30,
  'detect_threshold': 3,
  'detect_interval': 10,
  'freq_min': 300,
  'freq_max': 6000}}

In [39]:
recording_ids = (sgs.SpikeSortingRecordingSelection() & session_restrict & pfc_keys).fetch("recording_id")
sorter = "mountainsort4"
for recording_id in recording_ids:
    common_key = {
        "recording_id": recording_id,
        "sorter": sorter,
        "nwb_file_name": nwb_copy_file_name,
        "interval_list_name": str(
            (
                sgs.ArtifactDetectionSelection 
                & {"recording_id": recording_id}
            ).fetch1("artifact_id")
        ),
    } ## sgc.IntervalList contains intervals named by detected artifacts


    key = {
    **common_key,
    "sorter_param_name": "pfc_default",
}

    sgs.SpikeSortingSelection.insert_selection(key)

In [41]:
keys = (sgs.SpikeSortingSelection() & pfc_keys).fetch("KEY")
procs = smp.launch_workers((sgs.SpikeSorting), keys, n_workers=4)
for p in procs: p.join()

[2025-12-12 21:49:55,763][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,764][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,777][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,777][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,783][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,789][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,799][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:49:55,802][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:49:57,221][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:57,231][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:57,233][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:49:57,235][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-12 21:49:57,243][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-12 21:49:57,250][INFO]: Connected anirudh@172.16.102.154:3306

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[21:52:58][INFO] Spyglass: Writing new NWB file wilbur20210512_UA5URI90B0.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_UA5URI90B0.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmp8mi24zw4'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting:   0%|          | 0/1 [00:00<?, ?it/s][2025-12-12

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[21:55:27][INFO] Spyglass: Writing new NWB file wilbur20210512_FFFMTJ9ESW.nwb
[21:55:31][INFO] Spyglass: Writing new NWB file wilbur20210512_ORRWO1VXG8.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_FFFMTJ9ESW.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmpxo5puekn'>
  _warnings.warn(warn_message, R

...

/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_HCCS7SZZWK.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmp57kbye47'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting:   0%|          | 0/1 [00:00<?, ?it/s][2025-12-12 22:44:18,530][WARNING]: Skipped checksum for file with hash: d028b338-8431-9c

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording
Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[22:46:23][INFO] Spyglass: Writing new NWB file wilbur20210512_4RFU49MIM4.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_4RFU49MIM4.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmpr178intf'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting: 100%|██████████| 1/1 [04:19<00:00, 259.09s/it]
[

Mountainsort4 use the OLD spikeextractors mapped with NewToOldRecording


[22:52:24][INFO] Spyglass: Writing new NWB file wilbur20210512_KA98Y0K4LO.nwb
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/hash.py:39: ResourceWarning: unclosed file <_io.BufferedReader name='/media/labuser/NA_1_2025/spyglass/wilbur/analysis/wilbur20210512/wilbur20210512_KA98Y0K4LO.nwb'>
  return uuid_from_stream(Path(filepath).open("rb"), init_string=init_string)
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/site-packages/datajoint/external.py:281: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if check_hash:
/home/labuser/miniforge3/envs/spyglass/lib/python3.10/tempfile.py:869: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/media/labuser/NA_1_2025/spyglass/wilbur/tmp/tmphyczhmnj'>
  _warnings.warn(warn_message, ResourceWarning)
SpikeSorting: 100%|██████████| 1/1 [02:59<00:00, 179.35s/it]


In [42]:
sgs.SpikeSorting()

[2025-12-12 22:52:39,681][WARNING]: MySQL server has gone away. Reconnecting to the server.


sorting_id,analysis_file_name name of the file,object_id Object ID for the sorting in NWB file,"time_of_sort in Unix time, to the nearest second"
014c4043-4900-469b-85c3-4e2e96c9eb3e,J1620210722_TWG1PXM73Q.nwb,6b3d39e8-ec1d-4706-8d1a-b0db5173cce3,1762939225
039cacd1-def5-41c1-ba5d-12fbccc65033,J1620210722_MEPU6GNSIS.nwb,8d34636f-e34f-4c99-ba33-66a5ee2cd750,1762939509
04abb6b4-7cf8-408f-bce0-97baa216e6bd,wilbur20210512_VEL9SBH0M6.nwb,e47ee9f4-313f-4d40-90ce-160f17622985,1765559048
05274957-75da-4792-9b81-a045a5deec72,wilbur20210512_5LTKCZWWDL.nwb,5268d301-a794-43ef-b304-ceb67ba38b43,1765558606
0600316d-8b68-4411-a674-0d6f71c6f50e,wilbur20210512_CJHGUYLO0U.nwb,b6c82c9e-9f9f-4c0b-b0a3-6099fb74cfa0,1765557058
0e42a00b-2af3-424c-a308-f2322ca6b2eb,wilbur20210512_ORRWO1VXG8.nwb,9c0049f3-7ab0-4ba1-b53f-71d3b728e7f0,1765556715
0e814145-3137-4e99-aff8-c59ce673069f,J1620210722_4935GQ78JX.nwb,9efb67a8-a7fc-4044-b430-d8b029617476,1762838701
11d09ce8-1630-4fd9-b8c9-889f17c7940b,wilbur20210512_SKD2Z545GI.nwb,a0a31af0-3d05-4c49-851a-76ae5abf58f3,1765557812
18518ce3-19b6-442a-a48b-4abc14580543,J1620210722_5QGV0FBIK0.nwb,5aee29ac-b6c5-4d99-bc9c-76153caaa5d8,1762838588
1ac62da8-7c40-4377-a227-0e3c62b66f94,wilbur20210512_1TAY6RHOH8.nwb,70930bcc-e1c8-47aa-91eb-0d994f0f4868,1765558273


### Curation

#### Insert into CurationV1

In [43]:
sorting_ids = (sgs.SpikeSortingSelection & pfc_keys & session_restrict).fetch("sorting_id")
for sorting_id in sorting_ids:
    try:
        sgs.CurationV1.insert_curation(sorting_id = sorting_id)
    except KeyError:
        idx = sorting_id["sorting_id"]
        print(f"No units found in sorting {idx}!")

[22:54:15][INFO] Spyglass: Writing new NWB file wilbur20210512_1CL1MQUPUQ.nwb
[22:55:15][INFO] Spyglass: Writing new NWB file wilbur20210512_TA5TPAYSYI.nwb
[22:56:16][INFO] Spyglass: Writing new NWB file wilbur20210512_FWWSXWIWD6.nwb
[22:57:17][INFO] Spyglass: Writing new NWB file wilbur20210512_JM3WD95OZK.nwb
[22:58:18][INFO] Spyglass: Writing new NWB file wilbur20210512_FWZGJTFK56.nwb
[22:59:18][INFO] Spyglass: Writing new NWB file wilbur20210512_523HGUG95I.nwb
[23:00:19][INFO] Spyglass: Writing new NWB file wilbur20210512_MNZZRHD1O8.nwb
[23:01:20][INFO] Spyglass: Writing new NWB file wilbur20210512_TP42CWYU7G.nwb
[23:02:21][INFO] Spyglass: Writing new NWB file wilbur20210512_ABME4W0M1Q.nwb
[23:03:21][INFO] Spyglass: Writing new NWB file wilbur20210512_8RSB6YVORA.nwb
[23:04:21][INFO] Spyglass: Writing new NWB file wilbur20210512_GNLZD4EEVG.nwb
[23:05:22][INFO] Spyglass: Writing new NWB file wilbur20210512_C0VNOQ9PTA.nwb
[23:06:23][INFO] Spyglass: Writing new NWB file wilbur20210512_P

In [48]:
sgs.CurationV1() 

sorting_id,curation_id,parent_curation_id,analysis_file_name name of the file,object_id,merges_applied,description
014c4043-4900-469b-85c3-4e2e96c9eb3e,0,-1,J1620210722_VLYRZRZ48U.nwb,335df2aa-ffac-410c-a7f8-2bf12668f6ce,0,
014c4043-4900-469b-85c3-4e2e96c9eb3e,1,0,J1620210722_HYIE1SJ4IH.nwb,529764f3-06b7-495c-a629-ba0dafda862a,0,after metric curation
014c4043-4900-469b-85c3-4e2e96c9eb3e,2,1,J1620210722_4N1M5XMYJW.nwb,1ab74bf2-11e7-49b5-986a-e939a434862d,0,
039cacd1-def5-41c1-ba5d-12fbccc65033,0,-1,J1620210722_WMJIHCKMWY.nwb,3e1de769-564a-4b02-bebe-ef36448f253e,0,
039cacd1-def5-41c1-ba5d-12fbccc65033,1,0,J1620210722_ZMBP6BE4IL.nwb,d8650c20-294a-40db-8d79-a0f83b0c825c,0,after metric curation
039cacd1-def5-41c1-ba5d-12fbccc65033,2,1,J1620210722_ZQUAH2BABW.nwb,0ce23c4d-b8bb-4da4-b1dc-752729d5569f,0,
04abb6b4-7cf8-408f-bce0-97baa216e6bd,0,-1,wilbur20210512_523HGUG95I.nwb,24d648bf-b44f-49a3-a6e5-0e9dfe6b557b,0,
05274957-75da-4792-9b81-a045a5deec72,0,-1,wilbur20210512_HNEL79YCNG.nwb,916cc6d7-26d3-4297-8629-8ee7d976ee64,0,
0600316d-8b68-4411-a674-0d6f71c6f50e,0,-1,wilbur20210512_JVHLC83VGP.nwb,fd9fee3d-7e1f-4a4d-9db8-0a9b8659e3b4,0,
0e42a00b-2af3-424c-a308-f2322ca6b2eb,0,-1,wilbur20210512_L4MKUL2LWU.nwb,128d0a79-153f-43d5-9ea5-c528f7126990,0,


#### Round 1: Automatic Curation

In [50]:
label_params = {'nn_noise_overlap': ['>', 0.03, ['noise', 'reject']]}
merge_params = {}

sgs.MetricCurationParameters.insert1({"metric_curation_param_name": "noise_overlap_0.03",
                                      "label_params": label_params,
                                      "merge_params": merge_params
                                      }, skip_duplicates = True)

In [37]:
for sorting_idx in range(len((sgs.CurationV1() & session_restrict  & pfc_keys).fetch("sorting_id"))):
    key = {
        "sorting_id": (sgs.CurationV1() & session_restrict).fetch("sorting_id")[sorting_idx],
        "curation_id": 0,
        "waveform_param_name": "default_whitened",
        "metric_param_name": "franklab_default",
        "metric_curation_param_name": "noise_overlap_0.03",
    }
    sgs.MetricCurationSelection.insert_selection(key)
sgs.MetricCurationSelection()

[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43][WARNING] Spyglass: This row has already been inserted.
[16:48:43]

metric_curation_id,sorting_id,curation_id,waveform_param_name name of waveform extraction parameters,metric_param_name,metric_curation_param_name
02480e53-2868-4901-b26d-66e48f583d53,4f9c8ea6-bffb-476d-8908-a14f3fbe5aa7,0,default_whitened,franklab_default,noise_overlap_0.03
07b98aa8-2003-4f04-bab9-4c5085fa893a,430212fc-4265-480f-bed3-45387d7272d9,0,default_whitened,franklab_default,noise_overlap_0.03
0ec9b6d9-6f72-4121-9d55-9f8b90d24f04,7d115bb4-fdd3-4701-8369-e67eb32475ab,0,default_whitened,franklab_default,noise_overlap_0.03
117397f5-9d53-4923-b2d7-3651d116e931,589b0837-0196-41b2-984c-7f9855716898,0,default_whitened,franklab_default,noise_overlap_0.03
13e06c7c-e029-4f66-a06d-c71ae9505b05,fe96651e-adff-4bb3-9a23-0a2cb1eeab1c,0,default_whitened,franklab_default,noise_overlap_0.03
173402d5-01d7-473a-bb9f-6c2c8aaaf6e5,d83385c9-1e7c-480b-a3e0-58506057b775,0,default_whitened,franklab_default,noise_overlap_0.03
1e159fbe-72e7-46c6-9edf-6d41c38b6c13,b0482b2a-33b6-4964-b735-854c54cf6980,0,default_whitened,franklab_default,noise_overlap_0.03
1f8c96e4-bea2-4e57-aa5a-075fda265d83,f36656df-8942-4583-8a0e-f152dd36bc4e,2,default_whitened,franklab_default,pfc_round_2
2132d139-bb74-45e9-ab06-07e591b7be90,f3ca2fee-2b0a-4277-804f-f58b1dc0c3aa,0,default_whitened,franklab_default,noise_overlap_0.03
26ab8efb-fb29-4e2e-acc9-dfc7252bc15f,40cbdfef-5453-4022-9dcb-ca0abab6bed0,0,default_whitened,franklab_default,noise_overlap_0.03


In [46]:
keys = (sgs.MetricCurationSelection() - sgs.MetricCuration()).fetch("KEY")
procs = smp.launch_workers((sgs.MetricCuration), keys, n_workers=4)
for p in procs: p.join()

[2025-12-13 16:51:11,751][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,751][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,751][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,751][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,769][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,769][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,769][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-13 16:51:11,770][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-13 16:51:13,442][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:13,442][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:13,442][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:13,442][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-13 16:51:13,454][INFO]: Connected anirudh@172.16.102.154:3306
[2025-12-13 16:51:13,454][INFO]: Connected anirudh@172.16.102.154:330

In [47]:
sgs.MetricCuration()

[2025-12-14 14:05:50,495][WARNING]: MySQL server has gone away. Reconnecting to the server.


metric_curation_id,analysis_file_name name of the file,object_id Object ID for the metrics in NWB file
02480e53-2868-4901-b26d-66e48f583d53,J1620210722_TVYF18LDF6.nwb,ad72bebc-a0bb-4d54-b58a-0849e881223e
07b98aa8-2003-4f04-bab9-4c5085fa893a,J1620210722_A21KR53CUW.nwb,eb78fee5-dc8f-44d0-bd6b-52b32d93b3cf
0ec9b6d9-6f72-4121-9d55-9f8b90d24f04,J1620210722_VI5FPZC0Y6.nwb,24e7b6a6-cb5f-4d1a-b7d6-149139f4da0c
117397f5-9d53-4923-b2d7-3651d116e931,J1620210722_VEB5FXEJCG.nwb,7e80a9b0-d9a8-41f1-8f0b-10a0ce630bc8
13e06c7c-e029-4f66-a06d-c71ae9505b05,wilbur20210512_WGE8E2GXJH.nwb,94a780d1-b403-4d2c-97be-1c1d51ae5cce
173402d5-01d7-473a-bb9f-6c2c8aaaf6e5,J1620210722_N9N2QDLG9Z.nwb,73b0e071-5490-4060-8237-a669c268bd9e
1e159fbe-72e7-46c6-9edf-6d41c38b6c13,J1620210722_Z6LVJRF754.nwb,b2777ac5-d912-47c2-b306-ad8ea0f34d4b
1f8c96e4-bea2-4e57-aa5a-075fda265d83,J1620210722_MK40ZSE2EK.nwb,7edaf614-d0c2-4649-9207-f61fca95d6d1
2132d139-bb74-45e9-ab06-07e591b7be90,J1620210722_78MBASRWQO.nwb,de6abdc0-1a81-4d3a-851a-6255f41aaee0
26ab8efb-fb29-4e2e-acc9-dfc7252bc15f,wilbur20210512_XF4OPVOU0N.nwb,8535c994-6342-4c6d-bef9-811fc1117374


#### Round 2: Manual curation (FigURL)

In [48]:
keys = (sgs.MetricCurationSelection() & (sgs.SpikeSortingSelection() & pfc_keys & session_restrict)).fetch("KEY")

for key in keys:
    labels = sgs.MetricCuration.get_labels(key)
    merge_groups = sgs.MetricCuration.get_merge_groups(key)
    metrics = sgs.MetricCuration.get_metrics(key)
    sgs.CurationV1.insert_curation(
        sorting_id=(
            sgs.MetricCurationSelection
            & {"metric_curation_id": key["metric_curation_id"]}
        ).fetch1("sorting_id"),
        parent_curation_id=0,
        labels=labels,
        merge_groups=merge_groups,
        metrics=metrics,
        description="after metric curation",
    )

In [50]:
pfc_sorting_ids = (sgs.SpikeSortingSelection() & {"sorter_param_name": "pfc_default"} & session_restrict).fetch("KEY")
curation_keys = (sgs.CurationV1 &  pfc_sorting_ids & {"curation_id": 1}).fetch("KEY")
for key in curation_keys:
    # print("key: ", key)
    curation_uri = sgs.FigURLCurationSelection().generate_curation_uri(
        key
    )
    
    metric_curation_key = (sgs.MetricCurationSelection() & {"sorting_id": key["sorting_id"]} & session_restrict).fetch1("KEY")
    metrics = sgs.MetricCuration.get_metrics(metric_curation_key)
    # print("mc_key: ", metric_curation_key)
    # print("metrics: ", metrics.keys())
    
    fig_selection_key = {
        "sorting_id": key["sorting_id"],
        "curation_id": 1,
        "curation_uri": curation_uri,
        "metrics_figurl": list(metrics.keys())
    }
    sgs.FigURLCurationSelection.insert_selection(fig_selection_key)


sgs.FigURLCurationSelection()

figurl_curation_id,sorting_id,curation_id,curation_uri GitHub-based URI to a file to which the manual curation will be saved,metrics_figurl metrics to display in the figURL
008637bc-1444-4bcd-9817-a3839b0440c2,298552b8-9ebc-420d-b496-acb310a2e637,1,sha1://b8cedeb3172945d06d206fdde92902f85757896c,=BLOB=
1ceabd73-62cf-4cd6-bfa0-0b9cb086caa9,80021b2e-9bbc-4b87-825e-664ab9996717,1,sha1://e727cf3e3756dc32fdb27bbe514c392b024ddb0e,=BLOB=
723eee50-e53c-4969-b989-22c7906031eb,cf8a331d-6d33-4709-9ee7-ec50d8d61f4d,1,sha1://e557236c059e3d216e3ab837681af483b9fc3505,=BLOB=
913308e1-9cef-431f-b052-f47e1579ec3f,9e74fe06-2900-46e9-a56a-d6d79c2b8214,1,sha1://37e5ea18398f66d69f44bcefba17f6e4a1897e68,=BLOB=
92603573-e48d-4b32-abab-806f6ab520b8,4d12b6e6-4d16-4825-a80c-6189bedb22ef,1,sha1://34e92bf7179c6359adbad0db01fe38299ec39d25,=BLOB=
953257c7-3a9d-4f30-a662-b11ef5c70e4d,014c4043-4900-469b-85c3-4e2e96c9eb3e,1,sha1://a74325dfff69c2572bbbbf07b8418f66751d0638,=BLOB=
9ffa5a25-3e39-4400-b91b-4aeaf1bb4ed7,4e49d6f9-f067-4f33-8bfd-fd66505a614f,1,sha1://fedc906a9688d1554db500a6932f4ac4fc4143ad,=BLOB=
a7dbe5ef-d058-4433-a255-b011c90cb3d1,f3ca2fee-2b0a-4277-804f-f58b1dc0c3aa,1,sha1://48b705ae41d2b775cc907e24edbeb1c0cca77da5,=BLOB=
ccd43c4b-6a0b-4f93-8cae-27b89e37dc52,039cacd1-def5-41c1-ba5d-12fbccc65033,1,sha1://c2abfa053a1b9906339c4b2a620693287575bc3b,=BLOB=
d5abfc7b-3cd2-4b2a-8e90-328c51f3081d,f36656df-8942-4583-8a0e-f152dd36bc4e,1,sha1://1d3b7efd77ac806859719973a2bef65c10745e67,=BLOB=


In [51]:
keys =  (sgs.FigURLCurationSelection - sgs.FigURLCuration).fetch("KEY")
sgs.FigURLCuration.populate(keys)

[14:06:57][WARNING] Spyglass: Turning off transaction protection this table by default. Use use_transation=True to re-enable.
Read more about transactions:
https://docs.datajoint.io/python/definition/05-Transactions.html
https://github.com/LorenFrankLab/spyglass/issues/1030


In [52]:
sgs.FigURLCuration() # USE these urls to curate using the online interface #TODO figure out kachery

figurl_curation_id,url
008637bc-1444-4bcd-9817-a3839b0440c2,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:5abccf8d42a9183bfe3187de00deea83331c6560&s={""initialSortingCuration"":""sha1://b8cedeb3172945d06d206fdde92902f85757896c"",""sortingCuration"":""sha1://b8cedeb3172945d06d206fdde92902f85757896c""}&label=314d4d28-cf1e-4483-b741-a5e803be34df%20298552b8-9ebc-420d-b496-acb310a2e637&zone=default"
1ceabd73-62cf-4cd6-bfa0-0b9cb086caa9,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:a3fc054b04e5e7f033ceefbd98561e4befc7305f&s={""initialSortingCuration"":""sha1://e727cf3e3756dc32fdb27bbe514c392b024ddb0e"",""sortingCuration"":""sha1://e727cf3e3756dc32fdb27bbe514c392b024ddb0e""}&label=3acabea5-845c-4a8a-b2ae-88a35ce2e8e7%2080021b2e-9bbc-4b87-825e-664ab9996717&zone=default"
723eee50-e53c-4969-b989-22c7906031eb,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:2d1de5a5dddc81ea953b4b383452d7783581d134&s={""initialSortingCuration"":""sha1://e557236c059e3d216e3ab837681af483b9fc3505"",""sortingCuration"":""sha1://e557236c059e3d216e3ab837681af483b9fc3505""}&label=bed7c21f-eac7-4558-96e1-5e0b7e6692f6%20cf8a331d-6d33-4709-9ee7-ec50d8d61f4d&zone=default"
913308e1-9cef-431f-b052-f47e1579ec3f,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:eea9c027c7c45e6eeab6164041644ca9750eed11&s={""initialSortingCuration"":""sha1://37e5ea18398f66d69f44bcefba17f6e4a1897e68"",""sortingCuration"":""sha1://37e5ea18398f66d69f44bcefba17f6e4a1897e68""}&label=cfe28b8c-8b65-4118-a4e1-12c99fecc766%209e74fe06-2900-46e9-a56a-d6d79c2b8214&zone=default"
92603573-e48d-4b32-abab-806f6ab520b8,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:f6f759ac24699289df49052e631a27e5258e09ed&s={""initialSortingCuration"":""sha1://34e92bf7179c6359adbad0db01fe38299ec39d25"",""sortingCuration"":""sha1://34e92bf7179c6359adbad0db01fe38299ec39d25""}&label=2234ab9f-892c-4415-a35b-b0591a821b63%204d12b6e6-4d16-4825-a80c-6189bedb22ef&zone=default"
953257c7-3a9d-4f30-a662-b11ef5c70e4d,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:5c09eea2bbcc6a73b916d6ae42681aa3c18293a8&s={""initialSortingCuration"":""sha1://a74325dfff69c2572bbbbf07b8418f66751d0638"",""sortingCuration"":""sha1://a74325dfff69c2572bbbbf07b8418f66751d0638""}&label=f1aeaa05-8079-45bb-9bf9-52fc572ed599%20014c4043-4900-469b-85c3-4e2e96c9eb3e&zone=default"
9ffa5a25-3e39-4400-b91b-4aeaf1bb4ed7,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:d70b7b4941ca1415522b1cbeb27c6adb6f8bea4a&s={""initialSortingCuration"":""sha1://fedc906a9688d1554db500a6932f4ac4fc4143ad"",""sortingCuration"":""sha1://fedc906a9688d1554db500a6932f4ac4fc4143ad""}&label=e19837c7-6a06-40ff-ae44-5bce234c057b%204e49d6f9-f067-4f33-8bfd-fd66505a614f&zone=default"
a7dbe5ef-d058-4433-a255-b011c90cb3d1,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:5f0a54c83058b618ccc7897b79aa3b6ea88be8f4&s={""initialSortingCuration"":""sha1://48b705ae41d2b775cc907e24edbeb1c0cca77da5"",""sortingCuration"":""sha1://48b705ae41d2b775cc907e24edbeb1c0cca77da5""}&label=00ea2e2a-6705-4fdd-a223-f04f24089d2a%20f3ca2fee-2b0a-4277-804f-f58b1dc0c3aa&zone=default"
ccd43c4b-6a0b-4f93-8cae-27b89e37dc52,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:02e6eed994828a0a44dfcb3a141a4d75ccfe0252&s={""initialSortingCuration"":""sha1://c2abfa053a1b9906339c4b2a620693287575bc3b"",""sortingCuration"":""sha1://c2abfa053a1b9906339c4b2a620693287575bc3b""}&label=0fef4aa2-4df4-457f-90e7-f196791c3073%20039cacd1-def5-41c1-ba5d-12fbccc65033&zone=default"
d5abfc7b-3cd2-4b2a-8e90-328c51f3081d,"https://figurl.org/f?v=npm://@fi-sci/figurl-sortingview@12/dist&d=kachery:default:sha1:a60c5aeb5bf46424aa25e271c6a1ba09456f0f8f&s={""initialSortingCuration"":""sha1://1d3b7efd77ac8068597

#### Re-insert into sgs.CurationV1

In [ ]:
# DO THIS FOR EVERY PAIR OF initial_uri, updated_uri
initial_uri = ""
updated_uri = ""

sorting_id = (sgs.FigURLCurationSelection() &  {"curation_uri":initial_uri} & session_restrict).fetch1("sorting_id")
parent_curation_id = 1

labels = sgs.FigURLCuration.get_labels(updated_uri)
merge_groups = sgs.FigURLCuration.get_merge_groups(updated_uri)

curation_key = sgs.CurationV1.insert_curation(
    sorting_id=sorting_id,
    parent_curation_id= parent_curation_id,
    labels=labels,
    merge_groups=merge_groups,
    apply_merge=False,
)


#### Round 3: Automatic curation v2

In [ ]:
label_params = {'nn_noise_overlap': ['>', 0.03, ['noise', 'reject']],
                'isi_violation': ['>', 0.0025, ['noise', 'reject']],
                'nn_isolation': ['<', 0.95, ['noise', 'reject']],
                # 'Firing rate (Hz)': ['<', 0.1, ['noise', 'reject']],
                'snr': ['<', 1.5, ['noise', 'reject']]}

merge_params = {}

sgs.MetricCurationParameters().insert1({
    "metric_curation_param_name": "pfc_round_2",
    "label_params": label_params,
    "merge_params": merge_params
}, skip_duplicates = True)

In [ ]:
sgs.MetricCurationParameters().fetch()

In [ ]:
pfc_sorting_ids = (sgs.SpikeSortingSelection() & {"sorter_param_name": "pfc_default"}).fetch("KEY")
for sorting_idx in pfc_sorting_ids:
    key = {
        "sorting_id": sorting_idx["sorting_id"],
        "curation_id": 2,
        "waveform_param_name": "default_whitened",
        "metric_param_name": "franklab_default",
        "metric_curation_param_name": "pfc_round_2",
    }
    sgs.MetricCurationSelection.insert_selection(key)
sgs.MetricCurationSelection()

In [ ]:
keys = ((sgs.MetricCurationSelection() & {"curation_id": 2}) & (sgs.SpikeSortingSelection() & pfc_sorting_ids)).fetch("KEY")
procs = smp.launch_workers((sgs.MetricCuration), keys, n_workers=8)
for p in procs: p.join()

#### Downstream merge

In [24]:
from spyglass.spikesorting.spikesorting_merge import SpikeSortingOutput
from spyglass.spikesorting.analysis.v1.group import UnitSelectionParams
from spyglass.spikesorting.analysis.v1.group import SortedSpikesGroup
from spyglass.spikesorting.analysis.v1.unit_annotation import UnitAnnotation

In [ ]:
merge_insert_key = (sgs.CurationV1() & pfc_sorting_ids & {"curation_id":2} & session_restrict).fetch("KEY", as_dict = True)

SpikeSortingOutput.insert(merge_insert_key, part_name = "CurationV1")
SpikeSortingOutput()

In [ ]:
SortedSpikesGroup()

In [ ]:
## TODO add remaining stuff here ---